In [1]:
#evaluate de QA model by using the tutorial from https://haystack.deepset.ai/docs/latest/tutorial5md

# Step1: start ElasticSearch on localhost

# Step2: Fetch, Store And Preprocess the Evaluation Dataset

In [2]:
import os 

get_feedback_data = True

output_path = "data/tutorial5"

path_exists = os.path.exists(output_path)

if not path_exists:
    os.makedirs(output_path)

In [3]:
import subprocess
import json
import os

if (get_feedback_data == True):
    #url  = 'http://10.120.10.204:50960/api/feedback'
    url  = 'http://10.120.10.204:60022/api/feedback'

    #p = subprocess.call(['curl', '-X', 'GET', url, '-H', 'accept: application/json'])
    p = subprocess.check_output(['curl', '-X', 'GET', url, '-H', 'accept: application/json'])
    p = p.decode('UTF-8')

    json_res = json.loads(p)

    saved_file = output_path + "/json_res.json"
    with open(saved_file, 'w') as outfile:
        json.dump(json_res, outfile)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35270  100 35270    0     0  75686      0 --:--:-- --:--:-- --:--:-- 75686


In [4]:
#json_res

In [5]:
archive_already_fethced = True
if (archive_already_fethced == False):

    from haystack.utils import fetch_archive_from_http

    # Download evaluation data, which is a subset of Natural Questions development set containing 50 documents with one question per document and multiple annotated answers
    doc_dir = "data/tutorial5"
    s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/nq_dev_subset_v2.json.zip"

    fetch_archive_from_http(url=s3_url, output_dir=doc_dir)


In [6]:
# make sure these indices do not collide with existing ones, the indices will be wiped clean before data is inserted
doc_index = "test_docs"
label_index = "test_labels"

In [7]:
# Connect to Elasticsearch
from haystack.document_stores import ElasticsearchDocumentStore

# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(
    host="10.120.10.131",
    port="57664",
    #host="localhost",
    username="",
    password="",
    index=doc_index,
    label_index=label_index,
    embedding_field="emb",
    embedding_dim=768,
    excluded_meta_data=["emb"],
)

INFO - haystack.document_stores.base -  Numba not found, replacing njit() with no-op implementation. Enable it with 'pip install numba'.
2022-05-16 12:00:04.109992: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-16 12:00:04.110051: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/


In [8]:
output_full_path = output_path + "/json_squad_res.json"

if (get_feedback_data == True):
    with open(saved_file) as f:
        dataset = json.load(f)

    doc_counter = 1
    
    json_squad = {}
    json_squad["data"] = []
    
    #print(dataset)

    with open(output_full_path, "w") as f:

        for elem in dataset:
            
            curr_json_squad_elem = {}
            
            title = "title_" + str(doc_counter)
            paragraphs = []
            curr_paragraph = {}
            curr_paragraph["context"] = elem["answer"]["context"]

            qas = []

            curr_qa = {}
            curr_qa["answers"] = []

            curr_qa_curr_ans = {}

            curr_qa_curr_ans["answer_start"] = elem["answer"]["offsets_in_document"][0]["start"]
            curr_qa_curr_ans["text"] = elem["answer"]["answer"]

            curr_qa["answers"].append(curr_qa_curr_ans)

            curr_qa["id"] = "id_" + str(doc_counter)

            curr_qa["question"] = elem["query"]
            curr_qa["is_impossible"] = False

            qas.append(curr_qa)

            curr_paragraph["qas"] = qas

            paragraphs.append(curr_paragraph)
            
            curr_json_squad_elem["title"] = title
            curr_json_squad_elem["paragraphs"] = paragraphs
            
            json_squad["data"].append(curr_json_squad_elem)

            doc_counter += 1

        json.dump(json_squad, f, indent=4)


In [9]:
from haystack.nodes import PreProcessor
from haystack import Document

# Add evaluation data to Elasticsearch Document Store
# We first delete the custom tutorial indices to not have duplicate elements
# and also split our documents into shorter passages using the PreProcessor
preprocessor = PreProcessor(
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False,
)
document_store.delete_documents(index=doc_index)
document_store.delete_documents(index=label_index)



# The add_eval_data() method converts the given dataset in json format into Haystack document and label objects. Those objects are then indexed in their respective document and label index in the document store. The method can be used with any dataset in SQuAD format.
document_store.add_eval_data(
    #filename="data/tutorial5/nq_dev_subset_v3.json",
    #filename=saved_file,
    filename=output_full_path,
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor,
)

WARNING - haystack.document_stores.utils -  Could not convert an answer for 15 questions.
There were conversion errors for question ids: ['id_1', 'id_2', 'id_3', 'id_4', 'id_5', 'id_6', 'id_7', 'id_8', 'id_9', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_16']


# Step3: Initialize the Two Components of an ExtractiveQAPipeline: Retriever and Reader

In [10]:
# Initialize Retriever
from haystack.nodes import ElasticsearchRetriever

retriever = ElasticsearchRetriever(document_store=document_store)

# Alternative: Evaluate dense retrievers (EmbeddingRetriever or DensePassageRetriever)
# The EmbeddingRetriever uses a single transformer based encoder model for query and document.
# In contrast, DensePassageRetriever uses two separate encoders for both.

# Please make sure the "embedding_dim" parameter in the DocumentStore above matches the output dimension of your models!
# Please also take care that the PreProcessor splits your files into chunks that can be completely converted with
#        the max_seq_len limitations of Transformers
# The SentenceTransformer model "sentence-transformers/multi-qa-mpnet-base-dot-v1" generally works well with the EmbeddingRetriever on any kind of English text.
# For more information and suggestions on different models check out the documentation at: https://www.sbert.net/docs/pretrained_models.html

# from haystack.retriever import EmbeddingRetriever, DensePassageRetriever
# retriever = EmbeddingRetriever(document_store=document_store, model_format="sentence_transformers",
#                                embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1")
# retriever = DensePassageRetriever(document_store=document_store,
#                                   query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
#                                   passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
#                                   use_gpu=True,
#                                   max_seq_len_passage=256,
#                                   embed_title=True)
# document_store.update_embeddings(retriever, index=doc_index)

In [11]:
# Initialize Reader
from haystack.nodes import FARMReader

reader = FARMReader("deepset/roberta-base-squad2", top_k=4, return_no_answer=True)

# Define a pipeline consisting of the initialized retriever and reader
from haystack.pipelines import ExtractiveQAPipeline

pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

# The evaluation also works with any other pipeline.
# For example you could use a DocumentSearchPipeline as an alternative:

# from haystack.pipelines import DocumentSearchPipeline
# pipeline = DocumentSearchPipeline(retriever=retriever)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - backoff -  Backing off send_request(...) for 0.2s (requests.exceptions.ConnectTimeout: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f828f0188b0>, 'Connection to tm.hs.deepset.ai timed out. (connect timeout=15)'

# Step4: Evaluation of an ExtractiveQAPipeline

In [12]:
#Here we evaluate retriever and reader in open domain fashion on the full corpus of documents 
#i.e. a document is considered correctly retrieved if it contains the gold answer string within it. 
#The reader is evaluated based purely on the predicted answer string, regardless of which document this came from and the position of the extracted span.
#The generation of predictions is seperated from the calculation of metrics. 
#This allows you to run the computation-heavy model predictions only once and then iterate flexibly on the metrics or reports you want to generate.

In [13]:
from haystack.schema import EvaluationResult, MultiLabel

# We can load evaluation labels from the document storedocument_store
# We are also opting to filter out no_answer samples
eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=True)

## Alternative: Define queries and labels directly

# eval_labels = [
#    MultiLabel(
#        labels=[
#            Label(
#                query="who is written in the book of life",
#                answer=Answer(
#                    answer="every person who is destined for Heaven or the World to Come",
#                    offsets_in_context=[Span(374, 434)]
#                ),
#                document=Document(
#                    id='1b090aec7dbd1af6739c4c80f8995877-0',
#                    content_type="text",
#                    content='Book of Life - wikipedia Book of Life Jump to: navigation, search This article is
#                       about the book mentioned in Christian and Jewish religious teachings...'
#                ),
#                is_correct_answer=True,
#                is_correct_document=True,
#                origin="gold-label"
#            )
#        ]
#    )
# ]

# Similar to pipeline.run() we can execute pipeline.eval()
eval_result = pipeline.eval(labels=eval_labels, params={"Retriever": {"top_k": 5}})

Inferencing Samples:   0%|                          | 0/1 [00:00<?, ? Batches/s]/home/adi/anaconda3/envs/py384/lib/python3.8/site-packages/haystack/modeling/model/prediction_head.py:478: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████████████| 1/1 [00:00<00:00,  8.05 Batches/s]


In [14]:
eval_labels

[{'labels': [{'id': 'b49accb9-7ec3-4e4d-8599-9ec412711a2d', 'query': 'How to transition to urban environmental sustainability?', 'document': {'content': "Contents\n\nContents\n\nAcknowledgements .................................................................................................................... 4\n\nCOVID-19 preamble ..................................................................................................................... 5\n\nExecutive Summary .................................................................................................................... 7\n\n1 Introduction ......................................................................................................................... 10\n1.1 Background  ...........................................................................................................................10\n1.2 Methodological approach ..........................................................................................

In [15]:
# The EvaluationResult contains a pandas dataframe for each pipeline node.
# That's why there are two dataframes in the EvaluationResult of an ExtractiveQAPipeline.

retriever_result = eval_result["Retriever"]
retriever_result.head()

,multilabel_id,query,filters,gold_document_contents,content,gold_id_match,answer_match,gold_id_or_answer_match,rank,document_id,gold_document_ids,type,node,eval_mode,index
0,-5870905459667590399,How to transition to urban environmental sustainability?,b'null',[Contents\n\nContents\n\nAcknowledgements .....................................,Contents\n\nContents\n\nAcknowledgements ......................................,1.0,1.0,1.0,1.0,d9040e49f7351542b59643ad80d4e4a3-0,[d9040e49f7351542b59643ad80d4e4a3-0],document,Retriever,integrated,0
1,-5870905459667590399,How to transition to urban environmental sustainability?,b'null',[Contents\n\nContents\n\nAcknowledgements .....................................,Transport. Total transport emissions in 2020 have fallen by 6% compared to 2...,0.0,0.0,0.0,2.0,d2c47d238287e6a293d711d32b5c4d6b-0,[d9040e49f7351542b59643ad80d4e4a3-0],document,Retriever,integrated,1
2,-5870905459667590399,How to transition to urban environmental sustainability?,b'null',[Contents\n\nContents\n\nAcknowledgements .....................................,Several PFAS are sufficiently volatile to be considered long-range transboun...,0.0,1.0,1.0,3.0,b489faf3b80bae84dc72fb977c4ad17a-0,[d9040e49f7351542b59643ad80d4e4a3-0],document,Retriever,integrated,2
3,-5870905459667590399,How to transition to urban environmental sustainability?,b'null',[Contents\n\nContents\n\nAcknowledgements .....................................,Figure 14: Concentrations of BaP in 2019\n\nConcentrations of Benzo[a]pyrene...,0.0,1.0,1.0,4.0,393630a5645fb7c21d1300a1c97d3d9c-0,[d9040e49f7351542b59643ad80d4e4a3-0],document,Retriever,integrated,3


In [16]:
reader_result = eval_result["Reader"]
reader_result.head()

,multilabel_id,query,filters,gold_answers,answer,context,exact_match,f1,rank,document_id,gold_document_ids,offsets_in_document,gold_offsets_in_documents,type,node,eval_mode,index
0,-5870905459667590399,How to transition to urban environmental sustainability?,b'null',[the],,None,1.0,1.0,1.0,None,[d9040e49f7351542b59643ad80d4e4a3-0],"[{'start': 0, 'end': 0}]","[{'start': 1622, 'end': 1625}]",answer,Reader,integrated,0
1,-5870905459667590399,How to transition to urban environmental sustainability?,b'null',[the],94% were urban,"ns above 1.0 ng/m3 were registered at 27% of the stations, of which 94% were...",0.0,0.0,2.0,393630a5645fb7c21d1300a1c97d3d9c-0,[d9040e49f7351542b59643ad80d4e4a3-0],"[{'start': 260, 'end': 274}]","[{'start': 1622, 'end': 1625}]",answer,Reader,integrated,1
2,-5870905459667590399,How to transition to urban environmental sustainability?,b'null',[the],interview questions,... 66\n\nAppendix B Urban environmental sustainability transitions intervie...,0.0,0.0,3.0,d9040e49f7351542b59643ad80d4e4a3-0,[d9040e49f7351542b59643ad80d4e4a3-0],"[{'start': 3958, 'end': 3977}]","[{'start': 1622, 'end': 1625}]",answer,Reader,integrated,2
3,-5870905459667590399,How to transition to urban environmental sustainability?,b'null',[the],Key drivers of and barriers,.......................................................53\n4.2 Key drivers o...,0.0,0.0,4.0,d9040e49f7351542b59643ad80d4e4a3-0,[d9040e49f7351542b59643ad80d4e4a3-0],"[{'start': 3055, 'end': 3082}]","[{'start': 1622, 'end': 1625}]",answer,Reader,integrated,3


In [17]:
# We can filter for all documents retrieved for a given query
query = "who is written in the book of life"
retriever_book_of_life = retriever_result[retriever_result["query"] == query]

In [18]:
# We can also filter for all answers predicted for a given query
reader_book_of_life = reader_result[reader_result["query"] == query]

In [19]:
# Save the evaluation result so that we can reload it later and calculate evaluation metrics without running the pipeline again.
results_path = "./QAModelEvaluationResults/"

path_exists = os.path.exists(results_path)

if not path_exists:
    os.makedirs(results_path)

eval_result.save(results_path)

# Step5: Calculating Evaluation Metrics

In [20]:
#Load an EvaluationResult to quickly calculate standard evaluation metrics for all predictions, 
#such as F1-score of each individual prediction of the Reader node or recall of the retriever. 
#To learn more about the metrics, see Evaluation Metrics ( https://haystack.deepset.ai/guides/evaluation#metrics-retrieval )

In [21]:
saved_eval_result = EvaluationResult.load(results_path)
metrics = saved_eval_result.calculate_metrics()
print(f'Retriever - Recall (single relevant document): {metrics["Retriever"]["recall_single_hit"]}')
print(f'Retriever - Recall (multiple relevant documents): {metrics["Retriever"]["recall_multi_hit"]}')
print(f'Retriever - Mean Reciprocal Rank: {metrics["Retriever"]["mrr"]}')
print(f'Retriever - Precision: {metrics["Retriever"]["precision"]}')
print(f'Retriever - Mean Average Precision: {metrics["Retriever"]["map"]}')

print(f'Reader - F1-Score: {metrics["Reader"]["f1"]}')
print(f'Reader - Exact Match: {metrics["Reader"]["exact_match"]}')

Retriever - Recall (single relevant document): 1.0
Retriever - Recall (multiple relevant documents): 1.0
Retriever - Mean Reciprocal Rank: 1.0
Retriever - Precision: 0.75
Retriever - Mean Average Precision: 0.8055555555555555
Reader - F1-Score: 1.0
Reader - Exact Match: 1.0


# Step6: Generating an Evaluation Report

In [22]:
#A summary of the evaluation results can be printed to get a quick overview.
#It includes some aggregated metrics and also shows a few wrongly predicted examples.

In [23]:
pipeline.print_eval_report(saved_eval_result)

================== Evaluation Report ==================
                   Pipeline Overview
                      Query
                        |
                        |
                      Retriever
                        |
                        | recall_single_hit:   1.0
                        | recall_single_hit_top_1:   1.0
                        |
                      Reader
                        |
                        | exact_match:   1.0
                        | exact_match_top_1:   1.0
                        | f1:   1.0
                        | f1_top_1:   1.0
                        |
                      Output

                Wrong Retriever Examples
Query: 
 	How to transition to urban environmental sustainability?
Gold Document Ids: 
 	d9040e49f7351542b59643ad80d4e4a3-0
Metrics: 
 	recall_multi_hit: 1.0
 	recall_single_hit: 1.0
 	precision: 0.75
 	map: 0.8055555555555555
 	mrr: 1.0
 	ndcg: 0.9060254355346823
Documents: 
 	content: Contents

Contents

A

# Step7: Advanced Evaluation Metrics

In [24]:
#As an advanced evaluation metric, semantic answer similarity (SAS) can be calculated. 
#This metric takes into account whether the meaning of a predicted answer is similar to the annotated gold answer rather than just doing string comparison.
#To this end SAS relies on pre-trained models. 
#For English, we recommend "cross-encoder/stsb-roberta-large", whereas for German we recommend "deepset/gbert-large-sts". 
#A good multilingual model is "sentence-transformers/paraphrase-multilingual-mpnet-base-v2". 
#More info on this metric can be found in our paper ( https://arxiv.org/abs/2108.06130 ) 
#or in our blog post ( https://www.deepset.ai/blog/semantic-answer-similarity-to-evaluate-qa ).

In [25]:
advanced_eval_result = pipeline.eval(
    labels=eval_labels, params={"Retriever": {"top_k": 1}}, sas_model_name_or_path="cross-encoder/stsb-roberta-large"
)

metrics = advanced_eval_result.calculate_metrics()
print(metrics["Reader"]["sas"])

Inferencing Samples:   0%|                          | 0/1 [00:00<?, ? Batches/s]/home/adi/anaconda3/envs/py384/lib/python3.8/site-packages/haystack/modeling/model/prediction_head.py:478: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████████████| 1/1 [00:00<00:00,  2.15 Batches/s]
INFO - backoff -  Backing off send_request(...) for 1.3s (requests.exceptions.ConnectTimeout: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f838b614880>, 'Connection to tm.hs

0.1291466


# Step8: Isolated Evaluation Mode

In [26]:
#The isolated node evaluation uses labels as input to the Reader node instead of the output of the preceeding Retriever node. 
#Thereby, we can additionally calculate the upper bounds of the evaluation metrics of the Reader. 
#Note that even with isolated evaluation enabled, integrated evaluation will still be running.

In [27]:
eval_result_with_upper_bounds = pipeline.eval(
    labels=eval_labels, params={"Retriever": {"top_k": 5}, "Reader": {"top_k": 5}}, add_isolated_node_eval=True
)

Inferencing Samples:   0%|                          | 0/1 [00:00<?, ? Batches/s]/home/adi/anaconda3/envs/py384/lib/python3.8/site-packages/haystack/modeling/model/prediction_head.py:478: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████████████| 1/1 [00:00<00:00,  2.01 Batches/s]


In [28]:
pipeline.print_eval_report(eval_result_with_upper_bounds)

================== Evaluation Report ==================
                   Pipeline Overview
                      Query
                        |
                        |
                      Retriever
                        |
                        | recall_single_hit:   1.0
                        | recall_single_hit_top_1:   1.0
                        |
                      Reader
                        |
                        | exact_match upper bound:   1.0
                        | exact_match:   1.0
                        | exact_match_top_1:   1.0
                        | f1 upper bound:   1.0
                        | f1:   1.0
                        | f1_top_1:   1.0
                        |
                      Output

                Wrong Retriever Examples
Query: 
 	How to transition to urban environmental sustainability?
Gold Document Ids: 
 	d9040e49f7351542b59643ad80d4e4a3-0
Metrics: 
 	recall_multi_hit: 1.0
 	recall_single_hit: 1.0
 	precision: 0.75
 	m

# Step9: Evaluation of Individual Components: Retriever

In [29]:
## Evaluate Retriever on its own
# Note that no_answer samples are omitted when evaluation is performed with this method
retriever_eval_results = retriever.eval(top_k=5, label_index=label_index, doc_index=doc_index)
# Retriever Recall is the proportion of questions for which the correct document containing the answer is
# among the correct documents
print("Retriever Recall:", retriever_eval_results["recall"])
# Retriever Mean Avg Precision rewards retrievers that give relevant documents a higher rank
print("Retriever Mean Avg Precision:", retriever_eval_results["map"])

INFO - haystack.nodes.retriever.base -  Performing eval queries...
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.01it/s]
INFO - haystack.nodes.retriever.base -  For 1 out of 1 questions (100.00%), the answer was in the top-5 candidate passages selected by the retriever.


Retriever Recall: 1.0
Retriever Mean Avg Precision: 1.0


In [30]:
#Just as a sanity check, we can compare the recall from retriever.eval() with the multi hit recall from pipeline.eval(add_isolated_node_eval=True). 
#These two recall metrics are only comparable 
#since we chose to filter out no_answer samples when generating eval_labels and setting doc_relevance_col to "gold_id_match". 
#Per default calculate_metrics() has doc_relevance_col set to "gold_id_or_answer_match" 
#which interprets documents as relevant if they either match the gold_id or contain the answer.

In [31]:
metrics = eval_result_with_upper_bounds.calculate_metrics(doc_relevance_col="gold_id_match")
print(metrics["Retriever"]["recall_multi_hit"])

1.0


# Step10: Evaluation of Individual Components: Reader

In [35]:
#Here we evaluate only the reader in a closed domain fashion
#i.e. the reader is given one query and its corresponding relevant document 
#and metrics are calculated on whether the right position in this text is selected by the model as the answer span (i.e. SQuAD style)

In [36]:
# Evaluate Reader on its own
reader_eval_results = reader.eval(document_store=document_store, label_index=label_index, doc_index=doc_index)
top_n = reader_eval_results["top_n"]
# Evaluation of Reader can also be done directly on a SQuAD-formatted file without passing the data to Elasticsearch
# reader_eval_results = reader.eval_on_file("../data/nq", "nq_dev_subset_v2.json", device=device)

# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer including no_answers
print(f"Reader Top-{top_n}-Accuracy:", reader_eval_results["top_n_accuracy"])
# Reader Top-1-Exact Match is the proportion of questions where the first predicted answer is exactly the same as the correct answer including no_answers
print("Reader Top-1-Exact Match:", reader_eval_results["EM"])
# Reader Top-1-F1-Score is the average overlap between the first predicted answers and the correct answers including no_answers
print("Reader Top-1-F1-Score:", reader_eval_results["f1"])
# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer excluding no_answers
print(f"Reader Top-{top_n}-Accuracy (without no_answers):", reader_eval_results["top_n_accuracy_text_answer"])
# Reader Top-N-Exact Match is the proportion of questions where the predicted answer within the first n results is exactly the same as the correct answer excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-Exact Match (without no_answers):", reader_eval_results["top_n_EM_text_answer"])
# Reader Top-N-F1-Score is the average overlap between the top n predicted answers and the correct answers excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-F1-Score (without no_answers):", reader_eval_results["top_n_f1_text_answer"])

INFO - haystack.nodes.reader.farm -  Performing Evaluation using top_k_per_candidate = 3 
and consequently, QuestionAnsweringPredictionHead.n_best = 4. 
This deviates from FARM's default where QuestionAnsweringPredictionHead.n_best = 5
Evaluating: 100%|█████████████████████████████████| 1/1 [00:00<00:00,  2.00it/s]

Reader Top-4-Accuracy: 100.0
Reader Top-1-Exact Match: 0.0
Reader Top-1-F1-Score: 0.0
Reader Top-4-Accuracy (without no_answers): 100.0
Reader Top-4-Exact Match (without no_answers): 0.0
Reader Top-4-F1-Score (without no_answers): 15.384615384615385


In [37]:
#Just as a sanity check, we can compare the top-n exact_match 
#and f1 metrics from reader.eval() with the exact_match and f1 from pipeline.eval(add_isolated_node_eval=True). 
#These two approaches return the same values because pipeline.eval() calculates top-n metrics per default. 
#Small discrepancies might occur due to string normalization in pipeline.eval()'s answer-to-label comparison. reader.eval() does not use string normalization.

In [38]:
metrics = eval_result_with_upper_bounds.calculate_metrics(eval_mode="isolated")
print(metrics["Reader"]["exact_match"])
print(metrics["Reader"]["f1"])

1.0
1.0


INFO - backoff -  Backing off send_request(...) for 0.1s (requests.exceptions.ConnectTimeout: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f838b8af5b0>, 'Connection to tm.hs.deepset.ai timed out. (connect timeout=15)')))
ERROR - backoff -  Giving up send_request(...) after 4 tries (requests.exceptions.ConnectTimeout: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f828c1dff70>, 'Connection to tm.hs.deepset.ai timed out. (connect timeout=15)')))
ERROR - posthog -  error uploading: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f828c1dff70>, 'Connection to tm.hs.deepset.ai timed out. (connect timeout=15)'))
IN